In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Read in the data
lats = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lats.npy')
lons = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/lons.npy')

zl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/z400_11-15_12.npy')
zl_full = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/z400_11-15_12.npy')
zl_large = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_large/z400_11-15_12.npy')
zl_small = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_small/z400_11-15_12.npy')
zl_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_box/z400_11-15_12.npy')
zl_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_hole/z400_11-15_12.npy')

pvl_ctl = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/ctl/pv300-500_11-15_12.npy')
pvl_full = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_full/pv300-500_11-15_12.npy')
pvl_large = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_large/pv300-500_11-15_12.npy')
pvl_small = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_small/pv300-500_11-15_12.npy')
pvl_box = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_box/pv300-500_11-15_12.npy')
pvl_hole = np.load('/p/work1/lloveras/nov2018/proc_oct/4km_files/adj_hole/pv300-500_11-15_12.npy')

dpvl_full = pvl_full - pvl_ctl
dpvl_large = pvl_large - pvl_ctl
dpvl_small = pvl_small - pvl_ctl
dpvl_box = pvl_box - pvl_ctl
dpvl_hole = pvl_hole - pvl_ctl


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75

### Plot subdomain
min_lat = 29
max_lat = 45
min_lon = -95
max_lon = -79

### Cartopy projection
proj = crs.PlateCarree()

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],
                               ['bottomleft','bottommiddle','bottomright'],
                               ['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(7.2,5.4), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.left_labels=True

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('Control')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)
axd['topleft'].text(0.17, 0.55, 'No Perturbations',transform=axd['topleft'].transAxes + trans, fontsize=12,verticalalignment='top', bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
##############

# Add the gridlines
gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_box,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['topmiddle'].contourf(lons, lats, dpvl_box, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topmiddle'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Target')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_small,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['topright'].contourf(lons, lats, dpvl_small, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['topright'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Short $\lambda$')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
##############

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_full,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottomleft'].contourf(lons, lats, dpvl_full, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomleft'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('Full')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
##############

# Add the gridlines
gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_hole,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottommiddle'].contourf(lons, lats, dpvl_hole, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottommiddle'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('Target$^\complement$')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-92, -87, -82])
gls.ylocator = mticker.FixedLocator([32, 37, 42])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_ctl,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_large,10)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottomright'].contourf(lons, lats, dpvl_large, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',zorder=-20,
            transform=crs.PlateCarree())
axd['bottomright'].set_rasterization_zorder(-19)

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('Long $\lambda$')
axd['bottomright'].text(0.0, 1.0, '(f)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig11_wrf_4km_48h.pdf',bbox_inches='tight')
plt.show()
